#  Домашнее задание 3



In [22]:
import pandas as pd
import numpy as np
import requests
from tqdm.auto import tqdm
from scipy.stats import mode 
from pprint import pprint
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender
import warnings

from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP, MeanInvUserFreq, calc_metrics
from rectools.model_selection import TimeRangeSplitter

from userknn import UserKnn

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Датасет KION 

In [ ]:
%%time
!wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O ml-1m.zip
!unzip -o ml-1m.zip
!rm ml-1m.zip

In [6]:
interactions_df = pd.read_csv('../data_original/interactions.csv')
users = pd.read_csv('../data_original/users.csv')
items = pd.read_csv('../data_original/items.csv')

interactions_df.rename(columns={'last_watch_dt': Columns.Datetime,
                                'total_dur': Columns.Weight}, inplace=True) 
# will cast types and save new pd.DataFrame inside in Interactions.df
interactions = Interactions(interactions_df)   

# ! если хотите быстро прогнать этот ноутбук - раскомментируйте эту строку - она уменьшает данные
interactions = Interactions(interactions_df.sample(frac=0.01))  

interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
356872,369439,15719,2021-03-29,5113.0,100.0
2620403,474811,3076,2021-06-27,38.0,1.0
1686212,907268,9728,2021-07-29,2171.0,32.0
1180521,253876,3182,2021-06-22,6173.0,95.0
2502977,852967,6256,2021-07-03,9.0,0.0


# Создаем заготовку рекомендаций для холодных пользователей

In [13]:
max_date = interactions.df['datetime'].max()

train = interactions.df[(interactions.df['datetime'] < max_date - pd.Timedelta(days=7))]

In [14]:
from rectools.dataset import Dataset

dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=None,
    item_features_df=None
)

In [15]:
from rectools.models.popular import PopularModel

pop = PopularModel()
pop.fit(dataset)

In [17]:
pop_recs = pop.recommend(
    dataset.user_id_map.external_ids,
    dataset=dataset,
    k=10,
    filter_viewed=False  # True - удаляет просмотренные айтемы из рекомендаций 
)
pop_recs[['item_id', 'item_id', 'rank']].to_csv('../processed_data/popular_10_recs.csv')
pop_recs.head()

,user_id,item_id,score,rank
0,369439,10440,1911.0,1
1,369439,15297,1812.0,2
2,369439,13865,1144.0,3
3,369439,9728,1140.0,4
4,369439,4151,867.0,5


# Задаем фолды для кросс-валидации

In [18]:
N_SPLITS = 10
TEST_SIZE = '10D'

In [19]:
# Init generator of folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [20]:
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-05-15 00:00:00', freq='10D'),
  Timestamp('2021-05-25 00:00:00', freq='10D')),
 (Timestamp('2021-05-25 00:00:00', freq='10D'),
  Timestamp('2021-06-04 00:00:00', freq='10D')),
 (Timestamp('2021-06-04 00:00:00', freq='10D'),
  Timestamp('2021-06-14 00:00:00', freq='10D')),
 (Timestamp('2021-06-14 00:00:00', freq='10D'),
  Timestamp('2021-06-24 00:00:00', freq='10D')),
 (Timestamp('2021-06-24 00:00:00', freq='10D'),
  Timestamp('2021-07-04 00:00:00', freq='10D')),
 (Timestamp('2021-07-04 00:00:00', freq='10D'),
  Timestamp('2021-07-14 00:00:00', freq='10D')),
 (Timestamp('2021-07-14 00:00:00', freq='10D'),
  Timestamp('2021-07-24 00:00:00', freq='10D')),
 (Timestamp('2021-07-24 00:00:00', freq='10D'),
  Timestamp('2021-08-03 00:00:00', freq='10D')),
 (Timestamp('2021-08-03 00:00:00', freq='10D'),
  Timestamp('2021-08-13 00:00:00', freq='10D')),
 (Timestamp('2021-08-13 00:00:00', freq='10D'),
  Timestamp('2021-08-23 00:00:00', freq='10D'))]

## Задаем метрики и модели, по которым будем делать CV

In [23]:
# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    'map@10': MAP(k=10),
    'novelty': MeanInvUserFreq(k=10),
}

# few simple models to compare
models = {
    'cosine_userknn': CosineRecommender(), # implicit 
    'tfidf_userknn': TFIDFRecommender(), 
    'bm25_userknn': BM25Recommender()
}

# Тюнинг моделей Knn

In [ ]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.df.iloc[train_ids].copy()
    df_test = interactions.df.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=50)
        userknn_model.fit(df_train)
    
        recos = userknn_model.predict(df_test)
    
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)
        

# Метрики качества по фолдам 

In [25]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,map@10,novelty
0,0,cosine_userknn,0.000000,10.030283
1,0,tfidf_userknn,0.000000,10.059651
2,0,bm25_userknn,0.000000,10.059651
3,1,cosine_userknn,0.001634,9.850550
4,1,tfidf_userknn,0.001634,9.877150
5,1,bm25_userknn,0.001634,9.880053
6,2,cosine_userknn,0.000000,10.074047
7,2,tfidf_userknn,0.000000,10.081922
8,2,bm25_userknn,0.000000,10.090539
9,3,cosine_userknn,0.000667,9.849612


## Metrics mean 


In [26]:
df_metrics.groupby('model').mean()[metrics.keys()]

,map@10,novelty
model,,
bm25_userknn,0.000342,10.033808
cosine_userknn,0.000342,9.990610
tfidf_userknn,0.000314,9.994479


Выводы:
bm25 и cosine выдают более качественные рекомендации, чем tfidf, 

при этом bm25 предсказывает наиболее разнообразную выборку при лучшем из имеющихся скорах МАР

В итогом варианте выбираем bm25 модель

In [27]:
import pickle

with open('../models/model_bm25.pkl', 'wb') as f:
    pickle.dump(models['bm25_userknn'], f)

# Домашнее задание: максимум 20 баллов 

## Основные пункты оценки
1. значение метрики на лидерборде
2. ревью кода в ноутбуке 
3. реализация сервиса для модели


Вы можете сделать НЕ ВСЕ пункты и все равно получить 20 баллов. Получение > 20 баллов будет расцениваться как 20.

## Подробности

### 1. Побейте метрику на лидерборде map@10 = 0.063 для userKnn модели с семинара (5 баллов)

### 2. Реализуйте эксперименты с кастомной моделю kNN с семинара. Результат - ноутбук(и) (максимум 15 баллов)

(Вы можете отрефакторить код из userknn.py по желанию или не трогать его) 

Что можно сделать в ноутбуке:

- придумать, что делать с холодными пользователями в тесте. Сделайте рекомендации для них (обратите внимание на rectools.models.popular) (3 балла)

- сделать кол-во рекомендаций равным N, а не меньше N (3 балла)

- реализовать тюнинг гиперпараметров (например, векторного расстояния или типов kNN моделей (implicit/rectools/...)) и сделать выводы (3 балла)

- реализовать другие варианты ранжированивания айтемов похожих пользователей и сделать выводы (3 балла)

- провести эксперименты с параметрами оффлайн валидации и сделать выводы (3 балла)

### 3. Оберните модель в сервис (максимум 12 баллов)

- предпочтительный онлайн вариант: обучаете модель в ноутбуке, сохраняете обученную модель (pickle, dill), при запуске сервиса ее поднимаете и запрашиваете рекомендации "на лету" (12 баллов)
- или оффлайн вариант: предварительно посчитайте рекомендации для всех пользователей, сохраните и запрашивайте их (6 баллов)

### Хороший pull request - это:

- наличие описания (в идеале что сделано - по пунктам)

- код по стандарту PEP8
- легкая читаемость и воспроизводимость кода
- комментарии и объяснения. В ipynb пользуйтесь силой маркдауна. В скриптах пишите комментарии и докстринг.
- обоснование схемы валидации
- анализ метрики качества
